# Cython and NumPy, buffers, fused types

In [1]:
%load_ext Cython
import numpy as np

## Cython and numpy arrays -- original syntax

In [2]:
def shannon_entropy_py(p_x):
    return - np.sum(p_x * np.log(p_x))

In [3]:
%%cython

import numpy as np
cimport numpy as cnp

# https://en.wiktionary.org/wiki/Shannon_entropy

def shannon_entropy_cy(cnp.ndarray p_x):
    return - np.sum(p_x * np.log(p_x))
    

In [4]:
# entropy of the uniform distribution: log(n)

N = 1000
print(shannon_entropy_cy(np.ones((N,), dtype=np.float64) / N))
print(np.log(N))

# entropy of binomial distribution: 0.5 log(2 * pi * e * n * p * (1 - p))

from scipy.st

SyntaxError: invalid syntax (<ipython-input-4-d459153d558b>, line 9)

In [5]:
from scipy.stats import binom, uniform, poisson

In [6]:
n, mu = 100, 10.0
print(poisson.entropy(mu))
print(shannon_entropy_cy(poisson.pmf(np.arange(n), mu)))

2.5614099352749125
2.56140993527


In [7]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_cy(d)

6.96 µs ± 96.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_py(d)

6.92 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%%timeit poi=poisson(mu)
poi.entropy()

887 µs ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%cython

cimport cython
cimport numpy as cnp
from libc.math cimport log

@cython.boundscheck(False)
@cython.wraparound(False)
def shannon_entropy_faster(cnp.ndarray[double] p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * log(p_x[i])
    return -res

In [11]:
d = poisson.pmf(np.arange(n), mu)
(shannon_entropy_py(d), shannon_entropy_faster(d))

(2.5614099352749125, 2.5614099352749107)

In [12]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_faster(d)

2.91 µs ± 60.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%%cython

cimport cython
from libc.math cimport log

@cython.boundscheck(False)
@cython.wraparound(False)
def shannon_entropy_mv(double[::1] p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * log(p_x[i])
    return -res

In [14]:
d = poisson.pmf(np.arange(n), mu)
(shannon_entropy_py(d), shannon_entropy_mv(d))

(2.5614099352749125, 2.5614099352749107)

In [15]:
%%timeit d = poisson.pmf(np.arange(n), mu)
shannon_entropy_mv(d)

2.97 µs ± 59.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
